In [16]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from skimage import exposure, feature
import cv2

import tensorflow as tf 
import tensorflow.keras as keras 
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
import colorit
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")
K.set_floatx('float32')

In [17]:
TEST_DIR = "../../InternsAssignment/xraysdatabase/" # 0-class-1root/"
MODEL_DIR = "./7_fold_models/"
IMG_SHAPE = (230, 130)
NFOLDS = 5
BATCH_SIZE = 32

In [18]:
!ls -lh {MODEL_DIR}*.hdf5

-rw-r--r-- 1 jupyter jupyter 969K Nov 18 21:40 ./7_fold_models/fold_0_best_model.hdf5
-rw-r--r-- 1 jupyter jupyter 969K Nov 18 21:46 ./7_fold_models/fold_1_best_model.hdf5
-rw-r--r-- 1 jupyter jupyter 969K Nov 18 21:59 ./7_fold_models/fold_2_best_model.hdf5
-rw-r--r-- 1 jupyter jupyter 969K Nov 18 22:02 ./7_fold_models/fold_3_best_model.hdf5
-rw-r--r-- 1 jupyter jupyter 969K Nov 18 22:12 ./7_fold_models/fold_4_best_model.hdf5
-rw-r--r-- 1 jupyter jupyter 969K Nov 18 22:27 ./7_fold_models/fold_5_best_model.hdf5
-rw-r--r-- 1 jupyter jupyter 969K Nov 18 22:32 ./7_fold_models/fold_6_best_model.hdf5


In [19]:
# files = glob.glob(f'{TEST_DIR}/*/*.*')
# datafiles_df = pd.DataFrame(files, columns=["filename"])
# datafiles_df["class"] = [file.split("/")[-2] for file in files]
# datafiles_df["class"].value_counts()

In [20]:
datafiles_df = pd.read_csv("outputs/test.csv", index_col=0)

In [21]:
def equalize_hist(img):
    img_eq = exposure.equalize_hist(img, nbins=150)
    return img_eq

def adp_thresh(img):
    img = (img[:,:,0]*255).astype(np.uint8)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 131, 0)
    return img

def preprocessing_function(img):
    img_processed = equalize_hist(img)
    img_processed = np.expand_dims(adp_thresh(img_processed).astype("float32")*1.0/255.0,2)
    return img_processed

test_datagen = ImageDataGenerator(rescale=1./255, 
                                  preprocessing_function=preprocessing_function)

In [22]:
generator = test_datagen.flow_from_dataframe(
                datafiles_df,
#                 directory="../",
                shuffle=False,
                target_size=IMG_SHAPE,
                batch_size=BATCH_SIZE,
                color_mode="grayscale",
                class_mode='categorical')

Found 23 validated image filenames belonging to 2 classes.


In [24]:
pred = np.zeros((len(datafiles_df), 2))

model_files = glob.glob(f"{MODEL_DIR}*.hdf5")

with tf.device('/device:cpu:0'):
    for model_file in model_files:
        model = keras.models.load_model(model_file)

        pred_ = model.predict(generator, batch_size=32)
        pred += pred_ / len(model_files)
        

# Evaluation on Test Dataset

In [25]:
test = pd.get_dummies(datafiles_df["class"]).values
test_y = test[:,1]
pred_y = pred.argmax(1)

cr = classification_report(test_y, pred_y)
print(cr)

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.90      0.90      0.90        10

    accuracy                           0.91        23
   macro avg       0.91      0.91      0.91        23
weighted avg       0.91      0.91      0.91        23



In [26]:
correct_classified = (test_y == pred_y).sum()
total_samples = test_y.shape[0]
print(colorit.color_back(
    f"Accuracy: {np.round(correct_classified/total_samples, 2)}, {correct_classified}/{total_samples}\n" \
    f"Crossentropy {np.round(keras.metrics.categorical_crossentropy(test, pred).numpy().mean(), 4)}",
    255,255,0))


Accuracy: 0.91, 21/23
Crossentropy 0.2972
